In [2]:
import os
import re
import json
import xml.etree.ElementTree as ET

In [11]:
def replace_content(t):
    return re.sub(r'<bot name="([^"]+)"\s*/>', r'__\1__', t)

In [18]:
# path = './aiml-en-us-foundation-alice/ai.aiml'

In [18]:
data = []
for dirpath, _, filenames in os.walk('./aiml-en-us-foundation-alice/'):
    filenames = sorted([x for x in filenames if x.endswith('.aiml')])
    for filename in filenames:
        path = os.path.join(dirpath, filename)
        content = open(path).read()
        content = replace_content(content)
        root = ET.fromstring(content)
        for category in root.findall('category'):
            patterns = category.findall('pattern')
            templates = category.findall('template')
            for pattern in patterns:
                for template in templates:
                    if hasattr(pattern, 'text') \
                    and hasattr(template, 'text') \
                    and isinstance(pattern.text, str) \
                    and isinstance(template.text, str) \
                    and len(pattern.text) > 0 \
                    and len(template.text) > 0:
                        q = pattern.text.lower()
                        a = template.text
                        a = a.replace('\n', ' ')
                        entropy = 0
                        while '*' in q:
                            q = q.replace('*', '__entropy{}__'.format(entropy), 1)
                            entropy += 1
                        data.append((q, a, os.path.basename(path), True))
                        qq = re.sub(r'\s*__entropy[0-9]+__[\s\.]*$', '', q)
                        if qq != q:
                            data.append((qq, a, os.path.basename(path), False))

In [19]:
json.dump(data, open('alice.json', 'w'), indent=4, ensure_ascii=False)